<a href="https://colab.research.google.com/github/yuce-8/alpha/blob/main/YUCE_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YUCE-8.tech API documentation

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install requests

import json
import requests

## Connect to YUCE-8
this is subject to change in the next versions

In [4]:
class GCF():
  def __init__(self):
    #self.url = 'https://us-central1-yuce-8-v1.cloudfunctions.net/yuce_8_alpha_api'
    self.url = 'https://us-central1-yuce-8-v1.cloudfunctions.net/yuce_8_alpha_2_api'
    print('YUCE-8 ready @ ', self.url)

  def push(self, json_obj, return_result=False):
    r = requests.post(self.url, json=json_obj)
    if return_result:
      return r

yuce_8 = GCF()

YUCE-8 ready @  https://us-central1-yuce-8-v1.cloudfunctions.net/yuce_8_alpha_2_api


## Test connection
the test will return the current backend version

In [5]:
def get_current_backend_version():
   req = yuce_8.push({'action': 'get_version'}, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']
   else:
     return ';-( ' + req.json()['msg']


print(get_current_backend_version())

v0.1.2 alpha


## Create a test key
Key can be reused for 50 calls

In [6]:
def create_test_key(your_mail_address):
   req = yuce_8.push({'action': 'create_test_license', 'user_name': your_mail_address}, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']
   else:
     return ';-( ' + req.json()['msg']



key = create_test_key('torsten@yuce-8.info')
print('our new key = ', key)

our new key =  47902e26-1a51-4d5f-8e1e-2388f627ed57


## List all available forecasts

In [ ]:
def get_available_forecasts():
   req = yuce_8.push({'action': 'get_available_forecasts'}, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']
   else:
     return ';-( ' + req.json()['msg']

get_available_forecasts()

[{'BTC-USD': ['1d', '4hour']}]

## Get latest forecast

In [15]:
def get_latest_forecast(key, symbol, interval):
   req = yuce_8.push(
      {
        'action': 'get_forecast',
        'key': key,
        'symbol': symbol,
        'interval': interval
      }, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']['f_0']['yuce_8'], req.json()['msg']['api_calls_left_with_this_key'] 
   else:
     return ';-( ' + req.json()['msg'], -1

# get the forecast:
f_0, calls_left = get_latest_forecast(key, 'BTC-USD', '1d')

# print out the content:
if calls_left != -1:
  print('how many API calls are left?', calls_left)
  print('when was the forecast created?', f_0['forecast_created_at'])
  print('what is the last quote that was used to forecast?', f_0['last_quote'], ' @ ', f_0['last_date'])
  print('what is the name of the magic model that was used to forecast?', f_0['model_name'], 'last trained @', f_0['training_date'])
  print('what is the forecasted top range?', f_0['quotes_forecast_top'])
  print('what is the forecasted bottom range?', f_0['quotes_forecast_bottom'])

  # display the forecast timestep per timestep:
  print('timestep', '0 = ', f_0['last_quote'], ' @ ', f_0['last_date'])
  quote_0 = f_0['last_quote']
  top_range = f_0['quotes_forecast_top']
  bottom_range = f_0['quotes_forecast_bottom']

  for i in range(0, len(top_range)):
    forecast_top_at_i = top_range[i]
    forecast_bottom_at_i = bottom_range[i] 
    
    # see if it is a bearish or bullish forecast
    bearish = forecast_top_at_i <= quote_0

    print('timestep', i+1, 'should be w/in', ']' if bearish else '[', forecast_bottom_at_i, ', ', forecast_top_at_i, ']' if bearish else '[')

else:
  print(f_0)

how many API calls are left? 46
when was the forecast created? 2021-02-05 22:03:24.514608+01:00
what is the last quote that was used to forecast? 37472.08984375  @  2021-02-03 00:00:00
what is the name of the magic model that was used to forecast? model typ Y-2 last trained @ 2021-02-04 22:27:05
what is the forecasted top range? [37734.39447265625, 37734.39447265625, 37734.39447265625, 37734.39447265625, 37734.39447265625, 37734.39447265625, 37734.39447265625, 37734.39447265625, 37734.39447265625, 37734.39447265625, 37734.39447265625, 37734.39447265625, 37734.39447265625]
what is the forecasted bottom range? [37472.08984375, 37472.08984375, 37472.08984375, 37472.08984375, 37472.08984375, 37472.08984375, 37472.08984375, 37472.08984375, 37472.08984375, 37472.08984375, 37472.08984375, 37472.08984375, 37472.08984375]
timestep 0 =  37472.08984375  @  2021-02-03 00:00:00
timestep 1 should be w/in [ 37472.08984375 ,  37734.39447265625 [
timestep 2 should be w/in [ 37472.08984375 ,  37734.3944

## Backtesting the results

to be done